In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import os, requests, json, geojson
import pandas as pd
import geopandas as gpd
import ee
import geemap
import boto3

In [ ]:
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
# define directory
out_dir = os.getcwd()
bucket_name = 'cities-indicators'
aws_s3_dir = "https://"+bucket_name+".s3.eu-west-3.amazonaws.com"
boundary_ext = '/data/boundaries/'
indicators_file_aws = 'indicators/indicators.csv'

In [ ]:
OUTPUT_FILENAME = 'LND-1-permeable-areas.csv'

In [ ]:
# get list of cities
boundary_georef = pd.read_csv(aws_s3_dir + boundary_ext + 'boundary_georef.csv')
boundary_georef

In [ ]:
result = pd.DataFrame()
for i in range(len(boundary_georef)):    
    for boundary_name in ['aoi_boundary_name', 'units_boundary_name']:
        if type(boundary_georef.loc[i, boundary_name]) != float: # sometimes boundary_id is nan
            boundary_id = boundary_georef.loc[i, 'geo_name']+'-' + boundary_georef.loc[i, boundary_name]
            print(boundary_id)
            boundary_path = aws_s3_dir + boundary_ext +'boundary-'+boundary_id+'.geojson'
            boundary_geo = requests.get(boundary_path).json()
            ee_fc = geemap.geojson_to_ee(boundary_geo)
            ee_fc_witharea = ee_fc.map(lambda x: x.set({'area': x.geometry().area(1)}))

            imperv_areapixels = ee.Image("Tsinghua/FROM-GLC/GAIA/v10").pixelArea().multiply(ee.Image("Tsinghua/FROM-GLC/GAIA/v10").gt(0))
            imperv_area = imperv_areapixels.reduceRegions(
                collection=ee_fc,
                reducer=ee.Reducer.sum(),
                scale=30
            )

            general_areapixels = ee.ImageCollection("LANDSAT/LE07/C02/T1_TOA").first().pixelArea()
            general_area = general_areapixels.reduceRegions(
                collection=ee_fc,
                reducer=ee.Reducer.sum(),
                scale=30
            )

            df = geemap.ee_to_gdf(general_area)
            df['LND_1_percentPermeableSurface'] = 1 - (geemap.ee_to_gdf(imperv_area)['sum'] / df['sum'])
            result = pd.concat([result, df], axis=0)
            result[['geo_id', 'geo_level', 'geo_name', 'geo_parent_name', 'LND_1_percentPermeableSurface']].to_csv(OUTPUT_FILENAME)

In [ ]:
processedcities = pd.read_csv(OUTPUT_FILENAME)
processedcities

# Merge with indicator table

In [ ]:
# read indicator table
cities_indicators = pd.read_csv(aws_s3_dir +'/'+ indicators_file_aws)
cities_indicators

In [ ]:
def merge_indicators(indicator_table, new_indicator_table, indicator_name):
    if indicator_name in indicator_table.columns:
        print("replace with new calculations")
        indicator_table.drop(indicator_name, inplace=True, axis=1)
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    else:
        print("add new indicators")
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    return(cities_indicators_df)

In [ ]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators,
                                            new_indicator_table = processedcities,
                                            indicator_name = 'LND_1_percentPermeableSurface')

In [ ]:
cities_indicators_merged

## Upload in aws s3

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
# upload to aws
key_data = indicators_file_aws
cities_indicators_merged.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [ ]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')